In [2]:
import pandas as pd

In [78]:
df_predictions = pd.DataFrame({'start': [8, 17, 45, 56], 'end': [15, 25, 48, 61], 'type': ['PERSON', 'PERSON', 'DATE', 'DATE']})
df_targets = pd.DataFrame({'start': [10, 17, 45, 56, 80], 'end': [20, 25, 48, 61, 94], 'type': ['PERSON', 'PERSON', 'DATE', 'DATE', 'PERSON']})

In [79]:
df_predictions

,start,end,type
0,8,15,PERSON
1,17,25,PERSON
2,45,48,DATE
3,56,61,DATE


In [80]:
df_targets

,start,end,type
0,10,20,PERSON
1,17,25,PERSON
2,45,48,DATE
3,56,61,DATE
4,80,94,PERSON


In [82]:
df_predictions

,start,end,type,strict_pos
0,8,15,PERSON,8-15
1,17,25,PERSON,17-25
2,45,48,DATE,45-48
3,56,61,DATE,56-61


In [83]:
df_predictions.strict_pos

0     8-15
1    17-25
2    45-48
3    56-61
Name: strict_pos, dtype: object

In [84]:
df_targets

,start,end,type,strict_pos
0,10,20,PERSON,10-20
1,17,25,PERSON,17-25
2,45,48,DATE,45-48
3,56,61,DATE,56-61
4,80,94,PERSON,80-94


In [85]:
df_targets.strict_pos

0    10-20
1    17-25
2    45-48
3    56-61
4    80-94
Name: strict_pos, dtype: object

In [86]:
def compute_scores(predictions, targets, confusion_only=False):
    tp = 0
    fp = 0
    fn = 0
    targets = targets.copy()
    for prediction in predictions:
        if prediction in targets:
            tp += 1
            targets.remove(prediction)
        else:
            fp += 1
    fn += len(targets)
    
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    if confusion_only:
        return tp, fp, fn
    else:
        return {'precision': precision, 'recall': recall, 'f1': f1}

In [87]:
compute_scores(df_predictions.strict_pos.to_list(), df_targets.strict_pos.to_list())

{'precision': 0.75, 'recall': 0.6, 'f1': 0.6666666666666665}

In [90]:
for tipo in df_targets.type.unique():
    print(tipo)

PERSON
DATE


In [119]:
def compute_scores_document(df_predictions, df_targets):
    # strict calculation
    df_predictions['strict_pos'] = df_predictions.start.astype(str) + '-' + df_predictions.end.astype(str)
    df_targets['strict_pos'] = df_targets.start.astype(str) + '-' + df_targets.end.astype(str)

    strict_scores_dict = {}
    for entity_type in df_targets.type.unique():
        strict_scores_dict[entity_type] = compute_scores(df_predictions.loc[df_predictions.type == entity_type,'strict_pos'].to_list(), df_targets.loc[df_targets.type == entity_type, 'strict_pos'].to_list())

    # lenient calculation
    def overlap(a, b):
        if max(0, min(a[1], b[1]) - max(a[0], b[0])) != 0:
            return True
        else:
            return False

    lenient_scores_dict = {}
    for entity_type in df_targets.type.unique():
        prediction_lenient_match = []
        for pred_index, prediction in df_predictions.loc[df_predictions.type == entity_type].iterrows():
            for target_index, target in df_targets.loc[df_targets.type == entity_type].iterrows():
                prediction_pos = [prediction['start'], prediction['end']]
                target_pos = [target['start'], target['end']]
                if overlap(prediction_pos, target_pos):
                    prediction_lenient_match.append(target_index)
        prediction_lenient_match = list(set(prediction_lenient_match))
        lenient_scores_dict[entity_type] = compute_scores(prediction_lenient_match, df_targets.index.to_list())

    return {'strict': strict_scores_dict, 'lenient': lenient_scores_dict}

In [120]:
compute_scores_document(df_predictions, df_targets)

{'strict': {'PERSON': {'precision': 0.5,
   'recall': 0.3333333333333333,
   'f1': 0.4},
  'DATE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}},
 'lenient': {'PERSON': {'precision': 1.0,
   'recall': 0.4,
   'f1': 0.5714285714285715},
  'DATE': {'precision': 1.0, 'recall': 0.4, 'f1': 0.5714285714285715}}}

In [121]:
df_targets.reset_index(inplace=True)

In [98]:
df_targets['lenient_match'] = df_targets.apply(lambda row: 'match' + row['index'], axis=1)


In [111]:
def overlap(a, b):
    if max(0, min(a[1], b[1]) - max(a[0], b[0])) != 0:
        return True
    else:
        return False

In [112]:
prediction_lenient_match = []
for pred_index, prediction in df_predictions.iterrows():
    for target_index, target in df_targets.iterrows():
        prediction_pos = [prediction['start'], prediction['end']]
        target_pos = [target['start'], target['end']]
        if overlap(prediction_pos, target_pos):
            prediction_lenient_match.append(target_index)


In [114]:
list(set(prediction_lenient_match))

[0, 1, 2, 3]